In [1]:
import pandas as pd
import os
from scripts.utils_scenarios import build_scenarios

os.getcwd()

'/Users/ugo/Documents/MH2D_projets/michlougo/Ligue1'

In [2]:
df_emissions = pd.read_csv("./backend/data/calculated_travels/total_emissions.csv")

In [6]:
df_emissions

,visiting_team,hosting_team,transport_type,emissions_kg_co2,emissions_kg_co2_wo_empty_bus,travel_time_seconds,distance_km,alternative_emissions_kg_co2,alternative_emissions_kg_co2_wo_empty_bus,alternative_travel_time_seconds,alternative_distance_km,alternative_transport_type
0,Paris Saint-Germain,Paris Saint-Germain,/,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0,0.000,NaN
1,Monaco,Paris Saint-Germain,avion,16209.865726,13301.605726,17018.0,1523.655280,3213.670488,305.410488,48598,1750.264,train
2,Brest,Paris Saint-Germain,avion,12453.183654,10697.595654,12540.0,1117.809526,1912.461640,156.873640,29890,1080.256,train
3,Lille,Paris Saint-Germain,bus,680.817000,680.817000,18112.0,453.878000,803.285472,122.468472,13262,453.868,train
4,Nice,Paris Saint-Germain,avion,16062.700726,13221.319726,14042.0,1470.131280,3146.791488,305.410488,48598,1750.264,train
...,...,...,...,...,...,...,...,...,...,...,...,...
319,Nantes,Saint-Étienne,avion,12998.729888,11023.439888,12958.0,1205.642085,1975.290000,1975.290000,41456,1316.860,bus
320,Le Havre,Saint-Étienne,bus,2140.188000,2140.188000,47106.0,1426.792000,3452.475334,1312.287334,45998,1289.038,train
321,Auxerre,Saint-Étienne,avion,8178.218022,7096.253022,8010.0,577.403377,1081.965000,1081.965000,25544,721.310,bus
322,Angers,Saint-Étienne,avion,11040.032241,9322.502241,9814.0,915.601429,1717.530000,1717.530000,36100,1145.020,bus


In [3]:
import numpy as np

SCENARIO_TIME_LIMITS = {
    "SA": 0,
    "S-4h": 4 * 3600 * 2,  # *2 for aller retour
    "S-6h": 6 * 3600 * 2,  # *2 for aller retour
    "S-NP": np.inf,
}

scenarios_list_travels = build_scenarios(df_emissions, SCENARIO_TIME_LIMITS)

In [4]:
df_scenarios = pd.DataFrame(scenarios_list_travels)

In [5]:
# emissions per scenario per team
all_teams = set(df_scenarios["visiting_team"].unique().tolist()).union(
    df_scenarios["hosting_team"].unique().tolist()
)
print(all_teams)

{'Rennes', 'Strasbourg', 'Le Havre', 'Paris Saint-Germain', 'Lyon', 'Reims', 'Lens', 'Angers', 'Brest', 'Saint-Étienne', 'Nice', 'Montpellier', 'Auxerre', 'Nantes', 'Monaco', 'Lille', 'Marseille', 'Toulouse'}


In [ ]:
df_emissions_per_scenario_per_team_per_transport_type = pd.DataFrame()
for team in all_teams:
    df_team = df_scenarios[
        (df_scenarios["visiting_team"] == team) | (df_scenarios["hosting_team"] == team)
    ]
    df_team = (
        df_team.groupby(["scenario", "transport_type"])[
            [
                "emissions_kg_co2_from_transport",
                "emissions_kg_co2_from_empty_bus",
                "travel_time_seconds",
                "alternative_selected",
            ]
        ]
        .sum()
        .reset_index()
    )
    df_team.insert(0, "team", team)
    df_emissions_per_scenario_per_team_per_transport_type = pd.concat(
        [df_emissions_per_scenario_per_team_per_transport_type, df_team]
    )

df_emissions_per_scenario_per_team_per_transport_type[
    "emissions_tonnes_co2_from_transport"
] = (
    df_emissions_per_scenario_per_team_per_transport_type[
        "emissions_kg_co2_from_transport"
    ]
    / 1000
)

df_emissions_per_scenario_per_team_per_transport_type[
    "emissions_tonnes_co2_from_empty_bus"
] = (
    df_emissions_per_scenario_per_team_per_transport_type[
        "emissions_kg_co2_from_empty_bus"
    ]
    / 1000
)
df_emissions_per_scenario_per_team_per_transport_type["emissions_tonnes_co2"] = (
    df_emissions_per_scenario_per_team_per_transport_type[
        "emissions_tonnes_co2_from_empty_bus"
    ]
    + df_emissions_per_scenario_per_team_per_transport_type[
        "emissions_tonnes_co2_from_transport"
    ]
)
df_emissions_per_scenario_per_team_per_transport_type["travel_time_hours"] = (
    df_emissions_per_scenario_per_team_per_transport_type["travel_time_seconds"] / 3600
)
df_emissions_per_scenario_per_team_per_transport_type = (
    df_emissions_per_scenario_per_team_per_transport_type.drop(
        columns=[
            "emissions_kg_co2_from_transport",
            "emissions_kg_co2_from_empty_bus",
            "travel_time_seconds",
        ]
    )
)

In [73]:
df_emissions_per_scenario_per_transport_type = (
    df_emissions_per_scenario_per_team_per_transport_type.groupby(
        ["scenario", "transport_type"]
    )[
        [
            "emissions_tonnes_co2_from_transport",
            "emissions_tonnes_co2_from_empty_bus",
            "emissions_tonnes_co2",
            "travel_time_hours",
            "alternative_selected",
        ]
    ]
    .sum()
    .reset_index()
)

In [74]:
df_emissions_per_scenario_per_team_per_transport_type.to_csv(
    "./backend/data/calculated_travels/emissions_per_scenario_per_team_per_transport_type.csv",
    index=False,
)

In [ ]:
bus_sa = 101.495490 + 1050.125586 + 10.569066
bus_s4 = 106.600206 + 948.522630 + 108.806952
bus_s6 = 206.064930 + 359.852544 + 598.012314
bus_np = 277.894296 + 886.035492 + 359.852544
print(f"bus_sa: {bus_sa}")
print(f"bus_s4: {bus_s4}")
print(f"bus_s6: {bus_s6}")
print(f"bus_np: {bus_np}")


bus_sa: 1162.1901420000002
bus_s4: 1163.929788
bus_s6: 1163.929788
bus_np: 1523.782332


In [ ]:
df_emissions_per_scenario_per_transport_type.drop(["alternative_selected"], axis=1)[
    ~df_emissions_per_scenario_per_transport_type.scenario.str.contains("'")
]

,scenario,transport_type,emissions_tonnes_co2_from_transport,emissions_tonnes_co2_from_empty_bus,emissions_tonnes_co2,travel_time_hours
0,S-4h,avion,4768.706952,948.522630,5717.229582,1418.204444
1,S-4h,bus,106.600206,0.000000,106.600206,711.046667
2,S-4h,train,16.137560,108.806952,124.944512,490.280000
6,S-6h,avion,2825.735465,598.012314,3423.747779,808.271111
7,S-6h,bus,206.064930,0.000000,206.064930,1349.433333
8,S-6h,train,140.281372,359.852544,500.133916,1712.031111
12,S-NP,bus,277.894296,0.000000,277.894296,1831.017778
13,S-NP,train,411.187770,886.035492,1297.223262,4522.064444
16,SA,aller en bus\nretour en avion,0.000000,1.739646,1.739646,0.000000
17,SA,avion,5398.229111,1050.125586,6448.354697,1645.294444


In [ ]:
(
    df_emissions_per_scenario_per_transport_type.groupby(["scenario"])[
        [
            "emissions_tonnes_co2",
            "emissions_tonnes_co2_from_empty_bus",
            "emissions_tonnes_co2_from_transport",
            "travel_time_hours",
        ]
    ].sum()
).reset_index()

,scenario,emissions_tonnes_co2,emissions_tonnes_co2_from_empty_bus,emissions_tonnes_co2_from_transport,travel_time_hours
0,S-4h,5948.774300,1057.329582,4891.444718,2619.531111
1,S-4h',4891.444718,0.000000,4891.444718,2619.531111
2,S-6h,4129.946625,957.864858,3172.081767,3869.735556
3,S-6h',3172.081767,0.000000,3172.081767,3869.735556
4,S-NP,1575.117558,886.035492,689.082066,6353.082222
5,S-NP',689.082066,0.000000,689.082066,6353.082222
6,SA,6563.206351,1062.434298,5500.772053,2376.937778
7,SA',5500.772053,0.000000,5500.772053,2376.937778


In [ ]:
df_per_team = (
    df_emissions_per_scenario_per_team_per_transport_type.groupby(["scenario", "team"])[
        "emissions_tonnes_co2"
    ]
    .sum()
    .reset_index()
)

In [87]:
pivoted_df = pd.pivot_table(
    df_per_team, values="emissions_tonnes_co2", index="team", columns="scenario"
).sort_values(by="SA")

In [ ]:
pivoted_df["SA - S-4h"] = pivoted_df["SA"] - pivoted_df["S-4h"]

In [89]:
pivoted_df.to_clipboard()

In [90]:
pivoted_df

scenario,S-4h,S-4h',S-6h,S-6h',S-NP,S-NP',SA,SA',SA - S-4h
team,,,,,,,,,
Paris Saint-Germain,150.694065,103.703811,106.060391,59.070137,54.236182,7.245928,251.165211,209.862390,100.471146
Auxerre,239.450977,202.966837,140.559205,117.290011,53.985764,43.753070,289.416350,247.381937,49.965373
Reims,263.745596,216.722030,196.260506,147.596774,97.654961,48.991229,290.841444,246.820602,27.095847
Le Havre,276.061648,233.141230,216.363621,176.174613,75.754696,38.515798,299.270767,252.985768,23.209119
Angers,278.207446,229.289626,166.317017,124.669961,76.001851,34.354795,301.658953,250.300033,23.451507
Saint-Étienne,278.800196,231.057440,99.162289,66.404185,65.503959,30.605667,308.500479,259.675758,29.700283
Lyon,278.838475,226.380769,88.150195,41.563657,62.104308,15.517770,339.162364,287.645647,60.323889
Rennes,328.258372,269.634976,221.781392,166.049240,90.785409,39.286821,354.965380,297.392644,26.707007
Nantes,332.851211,279.457934,175.453129,131.086786,87.831297,40.969413,357.188995,302.633383,24.337784


In [ ]:
1 - (150.694065 / 251.165211)

0.4000201524724696